In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import pickle

import matplotlib.pyplot as plt

from sklearn.svm import LinearSVR
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [16]:
df_144 = pd.read_excel('./Datasets/Raw Data_Room 144.xlsx')

In [18]:
df_144.head(10)

,Time_0,AlarmSignal_0144,Time_1,OnOffState_0144,Time_2,OperationModeState_0144,Time_3,RoomTemp_0144,Time_4,SetTemp_0144,Time_5,OUTAIRHUMD,Time_6,OUTAIRTEMP
0,08.12.2017 15:33:07,NaN,08.12.2017 15:17:14,NaN,08.12.2017 15:33:26,NaN,08.12.2017 15:14:51,NaN,08.12.2017 15:18:08,NaN,08.12.2017 15:27:45,NaN,08.12.2017 15:12:37,NaN
1,08.12.2017 15:33:13,0.0,08.12.2017 15:17:24,1.0,08.12.2017 15:33:31,2.0,08.12.2017 15:14:56,19.6,08.12.2017 15:18:24,20.0,08.12.2017 15:27:46,62.180,08.12.2017 15:12:38,15.3
2,08.12.2017 15:43:14,0.0,08.12.2017 15:27:23,1.0,08.12.2017 15:43:31,2.0,08.12.2017 15:24:55,19.2,08.12.2017 15:28:23,20.0,08.12.2017 16:01:12,62.895,08.12.2017 16:01:12,15.1
3,08.12.2017 15:53:13,0.0,08.12.2017 15:37:23,1.0,08.12.2017 15:53:31,2.0,08.12.2017 15:34:55,19.2,08.12.2017 15:38:23,20.0,08.12.2017 16:47:20,63.935,08.12.2017 16:10:47,15.6
4,08.12.2017 16:03:13,0.0,08.12.2017 15:47:23,1.0,08.12.2017 16:03:31,2.0,08.12.2017 15:44:55,19.2,08.12.2017 15:48:23,20.0,08.12.2017 17:04:22,64.975,08.12.2017 16:29:39,15.1
5,08.12.2017 16:13:13,0.0,08.12.2017 15:57:23,1.0,08.12.2017 16:13:31,2.0,08.12.2017 15:54:55,19.6,08.12.2017 15:58:23,20.0,08.12.2017 17:18:10,66.015,08.12.2017 17:47:03,14.6
6,08.12.2017 16:23:13,0.0,08.12.2017 16:07:23,1.0,08.12.2017 16:23:31,2.0,08.12.2017 16:04:55,18.8,08.12.2017 16:08:23,20.0,08.12.2017 17:44:20,67.055,08.12.2017 19:13:42,14.1
7,08.12.2017 16:33:13,0.0,08.12.2017 16:17:23,1.0,08.12.2017 16:33:31,2.0,08.12.2017 16:14:55,18.8,08.12.2017 16:18:23,20.0,08.12.2017 18:17:26,68.095,09.12.2017 01:06:04,13.6
8,08.12.2017 16:43:13,0.0,08.12.2017 16:27:23,1.0,08.12.2017 16:43:31,2.0,08.12.2017 16:24:55,19.6,08.12.2017 16:28:23,20.0,08.12.2017 18:31:54,69.135,09.12.2017 03:12:37,13.6
9,08.12.2017 16:53:13,0.0,08.12.2017 16:37:23,1.0,08.12.2017 16:53:31,2.0,08.12.2017 16:34:55,19.2,08.12.2017 16:38:23,20.0,08.12.2017 19:17:26,70.175,09.12.2017 09:14:14,14.1


# 清理缺省值

In [19]:
def clean_NaN(df_Room,columns_Time,columns_Name):
    df_Room_Column =  pd.DataFrame(df_Room,columns = [columns_Time,columns_Name]).dropna(axis=0,how='any').reset_index().drop(['index'],axis=1)
    return df_Room_Column

In [20]:
df_144_Alarm = clean_NaN(df_144,'Time_0','AlarmSignal_0144')
df_144_State = clean_NaN(df_144,'Time_1','OnOffState_0144')
df_144_Mode = clean_NaN(df_144,'Time_2','OperationModeState_0144')
df_144_InTemp = clean_NaN(df_144,'Time_3','RoomTemp_0144')
df_144_SetTemp = clean_NaN(df_144,'Time_4','SetTemp_0144')
df_144_OutHum = clean_NaN(df_144,'Time_5','OUTAIRHUMD')
df_144_OutTemp = clean_NaN(df_144,'Time_6','OUTAIRTEMP')

# 清理重复值并修改日期形式

In [21]:
def transFormat_CleanDuplicated(df,column_names):
    df[column_names]=pd.to_datetime(df[column_names], format='%d.%m.%Y %H:%M:%S')
    df = df.drop_duplicates(column_names,keep ='first')
    df_Res = df.set_index(column_names)
    return df_Res

df_144_State_Res = transFormat_CleanDuplicated(df_144_State, 'Time_1')
df_144_Mode_Res = transFormat_CleanDuplicated(df_144_Mode, 'Time_2')
df_144_InTemp_Res = transFormat_CleanDuplicated(df_144_InTemp, 'Time_3')
df_144_SetTemp_Res = transFormat_CleanDuplicated(df_144_SetTemp, 'Time_4')
df_144_OutHum_Res = transFormat_CleanDuplicated(df_144_OutHum, 'Time_5')
df_144_OutTemp_Res = transFormat_CleanDuplicated(df_144_OutTemp, 'Time_6')


In [22]:
df_144_Mode_Res.head(10)

,OperationModeState_0144
Time_2,
2017-12-08 15:33:31,2.0
2017-12-08 15:43:31,2.0
2017-12-08 15:53:31,2.0
2017-12-08 16:03:31,2.0
2017-12-08 16:13:31,2.0
2017-12-08 16:23:31,2.0
2017-12-08 16:33:31,2.0
2017-12-08 16:43:31,2.0
2017-12-08 16:53:31,2.0


# 同步数据
## 以Mode作为主数据，再对齐其他数据


In [23]:
df_Task2 = df_144_Mode_Res
df_Task2['RoomTemp_0144'] = ''
df_Task2['SetTemp_0144'] = ''
df_Task2['OnOffState_0144'] = ''
df_Task2['OUTAIRHUMD'] = ''
df_Task2['OUTAIRTEMP'] = ''
df_144_Mode_Res.head()

,OperationModeState_0144,RoomTemp_0144,SetTemp_0144,OnOffState_0144,OUTAIRHUMD,OUTAIRTEMP
Time_2,,,,,,
2017-12-08 15:33:31,2.0,,,,,
2017-12-08 15:43:31,2.0,,,,,
2017-12-08 15:53:31,2.0,,,,,
2017-12-08 16:03:31,2.0,,,,,
2017-12-08 16:13:31,2.0,,,,,


# 重新设置索引并提取时间数据
-**提取为数组形式，加快数据处理速度**

In [24]:
df_Task2 = df_Task2.reset_index()
df_144_State_Res = df_144_State_Res.reset_index()
df_144_OutHum_Res = df_144_OutHum_Res.reset_index()
df_144_OutTemp_Res = df_144_OutTemp_Res.reset_index()
df_144_InTemp_Res = df_144_InTemp_Res.reset_index()
df_144_SetTemp_Res = df_144_SetTemp_Res.reset_index()

time_2 = df_Task2['Time_2'].tolist()
time_1 = df_144_State_Res['Time_1'].tolist()
time_5 = df_144_OutHum_Res['Time_5'].tolist()
time_6 = df_144_OutTemp_Res['Time_6'].tolist()
time_3 = df_144_InTemp_Res['Time_3'].tolist()
time_4 = df_144_SetTemp_Res['Time_4'].tolist()

Main_OnOff = df_Task2['OnOffState_0144'].tolist()
Main_OutHum = df_Task2['OUTAIRHUMD'].tolist()
Main_OutTemp = df_Task2['OUTAIRTEMP'].tolist()
Main_InTemp = df_Task2['RoomTemp_0144'].tolist()
Main_SetTemp = df_Task2['SetTemp_0144'].tolist()

time_1_OnOff = df_144_State_Res['OnOffState_0144'].tolist()
time_3_InTemp = df_144_InTemp_Res['RoomTemp_0144'].tolist()
time_4_SetTemp = df_144_SetTemp_Res['SetTemp_0144'].tolist()
time_5_OutHum = df_144_OutHum_Res['OUTAIRHUMD'].tolist()
time_6_OutTemp = df_144_OutTemp_Res['OUTAIRTEMP'].tolist()

# 转化时间为时间戳

In [25]:
for i in range(len(time_1)):
    time_1[i] = time_1[i].timestamp()
for i in range(len(time_2)):
    time_2[i] = time_2[i].timestamp()
for i in range(len(time_3)):
    time_3[i] = time_3[i].timestamp()
for i in range(len(time_4)):
    time_4[i] = time_4[i].timestamp()
for i in range(len(time_5)):
    time_5[i] = time_5[i].timestamp()
for i in range(len(time_6)):
    time_6[i] = time_6[i].timestamp()

# 对齐数据

In [26]:
# Onoff开关对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_1)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_1[j]) < 300:
            Main_OnOff[i] = time_1_OnOff[j]
            time_1_OnOff.pop(j)
            time_1.pop(j)
            break

In [27]:
#InTemp室温对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_3)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_3[j]) < 300:
            Main_InTemp[i] = time_3_InTemp[j]
            time_3_InTemp.pop(j)
            time_3.pop(j)
            break 

In [28]:
# SetTemp设温对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_4)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_4[j]) < 300:
            Main_SetTemp[i] = time_4_SetTemp[j]
            time_4_SetTemp.pop(j)
            time_4.pop(j)
            break
        

In [29]:
# OutHum室外湿度对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_5)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_5[j]) < 1800:
            Main_OutHum[i] = time_5_OutHum[j]
            break

In [30]:
# OutTemp室外温度对齐
for i in range(0,len(time_2)):
    for j in range(0,len(time_6)):
        # 三分钟以内的时间归位一类
        if abs(time_2[i] - time_6[j]) < 1800:
            Main_OutTemp[i] = time_6_OutTemp[j]
            break

In [31]:
df_Task2['RoomTemp_0144'] = Main_InTemp
df_Task2['SetTemp_0144'] = Main_SetTemp
df_Task2['OUTAIRHUMD'] = Main_OutHum
df_Task2['OUTAIRTEMP'] = Main_OutTemp
df_Task2['OnOffState_0144'] = Main_OnOff

In [32]:
df_Task2.head()

,Time_2,OperationModeState_0144,RoomTemp_0144,SetTemp_0144,OnOffState_0144,OUTAIRHUMD,OUTAIRTEMP
0,2017-12-08 15:33:31,2.0,19.2,20,1,62.18,15.3
1,2017-12-08 15:43:31,2.0,19.2,20,1,62.18,15.1
2,2017-12-08 15:53:31,2.0,19.6,20,1,62.18,15.1
3,2017-12-08 16:03:31,2.0,18.8,20,1,62.895,15.1
4,2017-12-08 16:13:31,2.0,18.8,20,1,62.895,15.1


In [33]:
df_Task2.to_csv('./Datasets/144_processed.csv',index = False)